# Orquestación Secuencial para Workflow de Gestión de Proyectos

## Descripción

En este ejercicio, dominarás el **patrón de orquestación secuencial** de Microsoft Agent Framework para crear un workflow colaborativo. Tres agentes especializados procesarán secuencialmente un problema o solicitud de proyecto: extracción de requisitos, evaluación de riesgos y planificación de acciones.

### Arquitectura Secuencial
```
Entrada de Proyecto
    ↓
[Requirements Extractor]
    ↓
[Risk Assessor]
    ↓
[Action Planner]
    ↓
Plan de Acción Priorizado
```

### Cargar librerías y variables de entorno

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from typing import cast
from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent
from agent_framework.openai import OpenAIChatClient

load_dotenv(find_dotenv(usecwd=True))

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

### Definir las Instrucciones de los Agentes

Cada agente tiene un rol específico en el flujo de trabajo: extractor de requisitos, evaluador de riesgos y planificador de acciones.

In [ ]:
extractor_instructions="""
Eres un Analista de Requisitos de Proyectos. Extrae de la entrada:
    - requisitos funcionales y no funcionales explícitos e implícitos
    - áreas o sistemas implicados (desarrollo, infraestructura, recursos humanos, marketing, finanzas)
    - restricciones de tiempo, presupuesto, recursos y regulaciones aplicables
Devuelve un resumen claro y objetivo en 1-2 frases.
"""

risk_instructions="""
Eres un Gestor de Riesgos de Proyectos.
Evalúa riesgos derivados del resumen: cumplimiento, impacto en recursos, plazos, costes y calidad.
Prioriza la criticidad (Alta/Media/Baja) y señala información faltante.
Devuelve 1 frase con la criticidad y 1 con los riesgos principales.
"""

planner_instructions="""
Eres un Planificador de Proyectos.
Propón la siguiente acción concreta en 1-2 frases: p.ej., análisis de viabilidad, reunión con stakeholders,
creación de cronograma, asignación de recursos, definición de métricas, revisión de presupuesto.
Incluye responsable sugerido (Equipo de Desarrollo/Infraestructura/RRHH/Marketing/Finanzas)."""

### Crear y ejecutar la orquestación secuencial

In [ ]:
async def process_project_request(prompt):
    # GitHub Models client configuration
    chat_client = OpenAIChatClient(
        model_id=MODEL_NAME,
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference"
    )
    
    extractor = chat_client.create_agent(
        instructions=extractor_instructions,
        name="requirements_extractor",
    )
    print("[1] Requirements extractor created")
    
    risk = chat_client.create_agent(
        instructions=risk_instructions,
        name="risk_assessor",
    )
    print("[2] Risk assessor created")
    
    planner = chat_client.create_agent(
        instructions=planner_instructions,
        name="action_planner",
    )
    print("[3] Action planner created")

    workflow = SequentialBuilder().participants([extractor, risk, planner]).build()

    outputs: list[list[ChatMessage]] = []
    print(f"\nProcessing project request: {prompt}")
    async for event in workflow.run_stream(f"Project input: {prompt}"):
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(list[ChatMessage], event.data))

    if outputs:
        print("\nOrchestration results:")
        for i, msg in enumerate(outputs[-1], start=1):
            if (msg.author_name is None):
                continue
            name = msg.author_name
            print(f"{'=' * 20} {name} {'=' * 20}\n{msg.text}\n")

# Enter project request here
print("Starting project request processing...")
user_prompt = "Necesitamos migrar nuestra aplicación web a una arquitectura de microservicios. Tenemos 6 meses de plazo y presupuesto limitado. ¿Plan?"
await process_project_request(user_prompt)

In [ ]:
# Enter project request here
print("Starting project request processing...")
user_prompt = "Queremos lanzar una campaña de marketing digital para aumentar ventas en un 30%. Necesitamos integración con CRM y analytics. Propón plan de validación."
await process_project_request(user_prompt)

In [ ]:
# Enter project request here
print("Starting project request processing...")
user_prompt = "Solicito implementar sistema de gestión documental con control de versiones y trazabilidad completa integrado con nuestro ERP actual."
await process_project_request(user_prompt)